In [1]:
!pip install pyspark findspark

     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 198 kB 18.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=2c6ab4b4b0d051add093c781507f48df27a1ea45715f14e13a1c537867b41c95
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
import findspark
findspark.init()
import pyspark

from pyspark import SparkContext
from pyspark.conf import SparkConf

conf = pyspark.SparkConf().set('spark.ui.port', '4050')
sc = SparkContext.getOrCreate()

----

In [4]:
from pyspark.sql import SparkSession


ss = SparkSession(sc).builder.config("spark.driver.memory", "6g").getOrCreate()

# weather_df = ss.read.option('header', True).csv("/gdrive/My Drive/aligned data/weather-ta2.csv").limit(20000)
# stocks_df = ss.read.option('header', True).csv('/gdrive/My Drive/aligned data/stocks-ta2.csv').limit(20000)
weather_df = sc.textFile("/gdrive/My Drive/aligned data/weather-ta2.csv")
stocks_df = sc.textFile("/gdrive/My Drive/aligned data/stocks-ta2.csv")
type(weather_df)

pyspark.rdd.RDD

In [12]:
# Clean headers
headerless_weather = weather_df.first() #extract header
weather = weather_df.filter(lambda row : row != headerless_weather).map(lambda line: line.split(","))
headerless_stocks = stocks_df.first() #extract header
stocks = stocks_df.filter(lambda row : row != headerless_stocks).map(lambda line: line.split(","))

In [13]:
weather = weather.map(lambda x: [x[1], x[2], x[3]]).take(20000)
stocks = stocks.map(lambda x: [x[1], x[2], x[3]]).take(20000)

In [14]:
type(weather)

list

----

In [8]:
#weather = sc.textFile('/content/drive/MyDrive/BigDataManagement/final data/weather-ta2.csv')
#header = weather.first()
#weather = weather.filter(lambda row: row != header)

In [15]:
weather = sc.parallelize(weather,4)
stocks = sc.parallelize(stocks,4)

In [16]:
#weather = weather.map(lambda row: row.split(',')[1:-1])
df1 = weather.keyBy(lambda row: row[1])
df1.take(5)

[('2016-01-01', ['01016099999', '2016-01-01', '4']),
 ('2016-01-02', ['01016099999', '2016-01-02', '6']),
 ('2016-01-03', ['01016099999', '2016-01-03', '0']),
 ('2016-01-04', ['01016099999', '2016-01-04', '0']),
 ('2016-01-05', ['01016099999', '2016-01-05', '-53'])]

In [17]:
type(df1)

pyspark.rdd.PipelinedRDD

In [18]:
df1.join(df1).take(5)

[('2016-01-08',
  (['01016099999', '2016-01-08', '-102'],
   ['01016099999', '2016-01-08', '-102'])),
 ('2016-01-08',
  (['01016099999', '2016-01-08', '-102'],
   ['02520099999', '2016-01-08', '-47'])),
 ('2016-01-08',
  (['01016099999', '2016-01-08', '-102'],
   ['02741099999', '2016-01-08', '-76'])),
 ('2016-01-08',
  (['01016099999', '2016-01-08', '-102'],
   ['02845099999', '2016-01-08', '-251'])),
 ('2016-01-08',
  (['01016099999', '2016-01-08', '-102'],
   ['03066099999', '2016-01-08', '37']))]

In [19]:
def comboid_create(row):
  el1 = row[1][0]
  el2 = row[1][1]
  comboid = [el1[0], el2[0]]
  comboid.sort()
  return ['_'.join(comboid), el1[1], el1[2], el2[2]]

df2 = df1.join(df1).filter(lambda row: row[1][0][0] > row[1][1][0]).map(comboid_create)
df2.take(5)

[['01016099999_02520099999', '2016-01-08', '-47', '-102'],
 ['01016099999_02741099999', '2016-01-08', '-76', '-102'],
 ['02520099999_02741099999', '2016-01-08', '-76', '-47'],
 ['01016099999_02845099999', '2016-01-08', '-251', '-102'],
 ['02520099999_02845099999', '2016-01-08', '-251', '-47']]

In [20]:
def max_x(a,b):
  if float(a) > float(b):
    return float(a)
  else:
    return float(b)

def min_x(a,b):
  if float(a) > float(b):
    return float(b)
  else:
    return float(a)

def avg(a,b):
  return (float(a) + float(b))/2

def aggregate(row):
  a = row[-2]
  b = row[-1]
  return [row[0], row[1], min_x(a,b), avg(a,b), max_x(a,b)]

df3 = df2.map(aggregate)
df3.take(5)

[['01016099999_02520099999', '2016-01-08', -102.0, -74.5, -47.0],
 ['01016099999_02741099999', '2016-01-08', -102.0, -89.0, -76.0],
 ['02520099999_02741099999', '2016-01-08', -76.0, -61.5, -47.0],
 ['01016099999_02845099999', '2016-01-08', -251.0, -176.5, -102.0],
 ['02520099999_02845099999', '2016-01-08', -251.0, -149.0, -47.0]]

combo_id, date, min, avg, max

-----

In [27]:
stocks.take(5)

[['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
  '2016-01-01',
  '57.5'],
 ['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
  '2016-01-02',
  '57.5'],
 ['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
  '2016-01-03',
  '57.5'],
 ['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
  '2016-01-04',
  '57.8'],
 ['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
  '2016-01-05',
  '57.17']]

In [21]:
stocks = stocks.map(lambda row: [row[0], row[1], float(row[2])]).keyBy(lambda row: row[1])

In [29]:
stocks.take(5)

[('2016-01-01',
  ['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
   '2016-01-01',
   57.5]),
 ('2016-01-02',
  ['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
   '2016-01-02',
   57.5]),
 ('2016-01-03',
  ['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
   '2016-01-03',
   57.5]),
 ('2016-01-04',
  ['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
   '2016-01-04',
   57.8]),
 ('2016-01-05',
  ['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
   '2016-01-05',
   57.17])]

-----

In [22]:
bigdata = stocks.join(df3.keyBy(lambda row: row[1]))
bigdata.take(1)

[('2016-01-06',
  (['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
    '2016-01-06',
    57.65],
   ['01016099999_02520099999', '2016-01-06', -104.0, -97.5, -91.0]))]

In [23]:
def parse_after_join(row):
  el1 = row[1][0]
  el2 = row[1][1]
  return [el1[0], el2[0], [el1[-1]], [el2[-3]], [el2[-2]], [el2[-1]]]

bigdata = bigdata.map(parse_after_join)
bigdata.first()

['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
 '01016099999_02520099999',
 [57.65],
 [-104.0],
 [-97.5],
 [-91.0]]

(stock_id, combo_id, price, min, avg, max)

In [24]:
agg = bigdata.keyBy(lambda row: (row[0], row[1])).reduceByKey(lambda a,b: [a[-4] + b[-4], a[-3] + b[-3],a[-2] + b[-2],a[-1] + b[-1]])

In [25]:
import numpy as np

def cosine(a, b):
  return float(np.dot(np.array(a), np.array(b)) / (np.linalg.norm(np.array(a)) * np.linalg.norm(np.array(b))))

In [26]:
def parser(key):
  return key[:-12], key[-11:]

In [27]:
def get_cosine(tuple_row):
  price_min = cosine(tuple_row[1][0],tuple_row[1][1])
  price_avg = cosine(tuple_row[1][0],tuple_row[1][2])
  price_max = cosine(tuple_row[1][0],tuple_row[1][3])
  stock, combo_id = tuple_row[0]
  station1, station2 = parser(combo_id)
  return [stock, station1, station2, price_min, price_avg, price_max]

In [28]:
results = agg.map(get_cosine)

In [29]:
tau=[0.90,0.90,0.90]

min_results = results.filter(lambda row: row[-3]> tau[0]).collect()
avg_results = results.filter(lambda row: row[-2]> tau[1]).collect()
max_results = results.filter(lambda row: row[-1]> tau[2]).collect()


In [45]:
avg_results

[['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
  '02520099999',
  '08524099999',
  0.6818570512389417,
  0.9395834323743936,
  0.9797305490547166],
 ['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
  '03862099999',
  '08524099999',
  0.8585219230512734,
  0.9599359426120972,
  0.9797689496049381],
 ['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
  '08524099999',
  '10488099999',
  0.7477140148544775,
  0.9387384179318948,
  0.97926929786051],
 ['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
  '08524099999',
  '11120099999',
  0.6949079257395908,
  0.9354724616570542,
  0.9798146618002246],
 ['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
  '08524099999',
  '11150099999',
  0.7114471190587496,
  0.9347348553613

(stock_name, station_id_1, station_id_2, price_min_cossim, price_avg_cossim, price_max_cossim)

In [30]:
print("Results =>")
print(f"Min Results(count={len(min_results)}): ",min_results)
print(f"Avg Results(count={len(avg_results)}): ",avg_results)
print(f"Max Results(count={len(max_results)}): ",max_results)

Results =>
Min Results(count=6):  [['1051.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Goodman-Group_011495', '06207099999', '08524099999', 0.9038653426772594, 0.9613800252089494, 0.978528660756664], ['10612.Asien--Australien--Südamerika--Afrika_Japan_Techno-Alpha_0492WS', '06207099999', '08524099999', 0.9006077362167587, 0.964636354070597, 0.9860404654101109], ['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404', '06207099999', '08524099999', 0.909122639089026, 0.9641315701974623, 0.9795555350139057], ['10698.Asien--Australien--Südamerika--Afrika_Japan_Tiemco-Ltd._0493AO', '06207099999', '08524099999', 0.9026194377133125, 0.964841150386873, 0.985040564938908], ['10679.Asien--Australien--Südamerika--Afrika_Japan_The-Michinoku-Bank_0490JR', '06207099999', '08524099999', 0.9046139361446396, 0.9680027404971031, 0.9889081391030721], ['1081.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Infigen-Energy-Stap._01126U', '06

In [ ]:
results.sortBy(lambda row: row[-2], ascending=False).take(10)

[['10679.Asien--Australien--Südamerika--Afrika_Japan_The-Michinoku-Bank_0490JR',
  '06207099999',
  '08524099999',
  0.9046139361446396,
  0.9680027404971031,
  0.9889081391030721],
 ['10679.Asien--Australien--Südamerika--Afrika_Japan_The-Michinoku-Bank_0490JR',
  '03066099999',
  '08524099999',
  0.8502761184041757,
  0.9667747023429623,
  0.989104791777618],
 ['10698.Asien--Australien--Südamerika--Afrika_Japan_Tiemco-Ltd._0493AO',
  '06207099999',
  '08524099999',
  0.9026194377133125,
  0.964841150386873,
  0.985040564938908],
 ['10612.Asien--Australien--Südamerika--Afrika_Japan_Techno-Alpha_0492WS',
  '06207099999',
  '08524099999',
  0.9006077362167587,
  0.964636354070597,
  0.9860404654101109],
 ['10679.Asien--Australien--Südamerika--Afrika_Japan_The-Michinoku-Bank_0490JR',
  '03862099999',
  '08524099999',
  0.8536509536957989,
  0.9641612314038129,
  0.9890492205640301],
 ['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
  